## Contextual Chatbot:
It is a type of conversational artificial intelligence that is designed to understand the context of the conversation and respond accordingly. The chatbot uses the context of the conversation to determine the user's intent and generate an appropriate response.


### Tflearn
TFLearn can be defined as a modular and transparent deep learning aspect used in TensorFlow framework. 
The main motive of TFLearn is to provide a higher level API to TensorFlow for facilitating and showing up new experiments.


### Why Tflearn over Keras ?
    1. With Tflearn we can use arrays directly. On other hand, in keras it needs numpy arrays.
    2. Tflearn API is more closer to Tensorflow code as compared to keras.
    3. Tflearn provides better performance than keras.

### Intent
intent refers to the goal the customer has in mind when typing in a question or comment.
Intent are very important component of the chatbot platform

In [61]:
# Labraries needed for NLP:

import nltk
from nltk.stem.lancaster import LancasterStemmer   # It convert any word to its root word
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stemmer = LancasterStemmer()

In [62]:
# Labraries need for Tensorflow processing:

import tensorflow as tf
import numpy as np
import tflearn
import random
import json

In [63]:
# Importing the dataset

with open('intents.json') as json_data:
    intents = json.load(json_data)
    
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'hours',
   'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of Lon

In [64]:
words = []
classes = []
documents = []
ignore = ["?", "!", "$"]

for intent in intents['intents']:  # loop through each sentence in intent's pattern
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)  # tokenizing each and every word in the sentence
        words.extend(w)
        documents.append((w, intent['tag']))  # Storing values with their keys(tags) in the list of tuple
        
        if intent['tag'] not in classes:  # storing only keys(tags) in this.
            classes.append(intent['tag'])

In [65]:
print("Words is the list of pattern key in the dataset: \n{} \n\nClasses is the tags: \n{} \n\nDocuments is the list of list which stores values and keys: \n{}".format(words, classes, documents))

Words is the list of pattern key in the dataset: 
['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'What', 'hours', 'are', 'you', 'open', '?', 'What', 'are', 'your', 'hours', '?', 'When', 'are', 'you', 'open', '?', 'What', 'is', 'your', 'location', '?', 'Where', 'are', 'you', 'located', '?', 'What', 'is', 'your', 'address', '?', 'Where', 'is', 'your', 'restaurant', 'situated', '?', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Are', 'you', 'cash', 'only', '?', 'What', 'is', 'your', 'menu', 'for', 'today', '?', 'What', 'are', 'you', 'serving', 'today', '?', 'What', 'is', 'today', "'s", 'special', '?', 'Do', 'you', 'provide', 'home', 'delivery', '?', 'Do', 'you', 'deliver', 'the', 'food', '?', 'What', 'are', 'the', 'home', 'delivery', 'options', '?', 'What', 'is', 'your', 'Menu', '?', 'What', 'are', 'the', 'main', 'course',

In [66]:
print(type(documents[1]))

<class 'tuple'>


In [67]:
# Perform stemming and lower each word as well as remove duplicates

words = [stemmer.stem(str(w).lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words\t", words)

31 documents
9 classes ['deliveryoption', 'goodbye', 'greeting', 'hours', 'location', 'menu', 'payments', 'thanks', 'todaysmenu']
57 unique stemmed words	 ["'s", 'acceiv', 'address', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'cours', 'credit', 'day', 'del', 'delicy', 'delivery', 'dish', 'do', 'food', 'for', 'from', 'good', 'goodby', 'hello', 'help', 'hi', 'hom', 'hour', 'how', 'is', 'lat', 'loc', 'main', 'mastercard', 'me', 'menu', 'most', 'on', 'op', 'opt', 'provid', 'resta', 'see', 'serv', 'situ', 'spec', 'tak', 'tel', 'thank', 'that', 'the', 'ther', 'today', 'what', 'when', 'wher', 'yo', 'you']


### Creating the training data

In [68]:
training = []
output = []

# Creating an empty array for output
op_empty = [0]*len(classes)

# Creating training set, bags of words for each sentence
for doc in documents:
    bag = []   # Initalizing bag of words
    pattern_words = doc[0]  # List of tokenizised words for the pattern
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  # Stemming each word
    
    for w in words:  # creating bag of words array
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # Output is '1' for current tag and '0' for rest of other tags
    output_row = list(op_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

C:\Users\acer\AppData\Local\Temp\ipykernel_11992\742703838.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [69]:
# Splitting the training into x and y train

x_train = list(training[:,0])
y_train = list(training[:,1])

In [70]:
print(x_train, y_train)

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## Creating the Neural Network from scratch

In [71]:
# Resetting the underlying graph
tf.compat.v1.reset_default_graph()

1. net = tflearn.input_data(shape=[None, len(x_train[0])]):  
This line creates an input layer for the network and specifies the shape of the input data. The shape parameter takes a list of integers that define the shape of the input data. The first dimension, None, represents the number of samples in the data, which can be of any size. The second dimension, len(x_train[0]), represents the number of features in each sample, which is the number of words in the pattern.
 
  
2. net = tflearn.fully_connected(net, 10):  
This line creates a fully connected layer in the network with 10 neurons. The first argument, net, is the input to the layer, which is the output from the previous layer. The second argument, 10, is the number of neurons in the layer.

  
3. net = tflearn.fully_connected(net, 10):   
This line creates another fully connected layer with 10 neurons. The input to this layer is the output from the previous layer.

  
4. net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax'):  
This line creates the final fully connected layer with len(y_train[0]) neurons. The activation function used in this layer is softmax, which is a common activation function used in the output layer of a neural network for classification tasks. The output of the softmax function is a probability distribution over the possible classes.

  
5. net = tflearn.regression(net): 
This line creates a regression model in TFLearn. The input to the model is the output from the previous layer. A regression model

In [72]:
# Building the neural_network

net = tflearn.input_data(shape=[None, len(x_train[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
net = tflearn.regression(net)

In [73]:
#  Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir = 'tflearn_logs')   # DNN =  Deep learning Neural Network

In [74]:
# Starting training the model

model.fit(x_train, y_train, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.12333 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.12333 - acc: 0.9891 -- iter: 24/31
Training Step: 4000  | total loss: 0.11586 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.11586 - acc: 0.9902 -- iter: 31/31
--
INFO:tensorflow:C:\Users\acer\Deep Learning\Chat bot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


#### Pickle module:
The pickle module in Python is used to serialize and deserialize Python objects, which means it can be used to save a trained machine learning or deep learning model to disk and load it back later to make predictions.

In deep learning, training a model can be a time-consuming and computationally expensive task. After training the model, we may want to save it to avoid having to train it again from scratch each time we want to make predictions. The pickle module can be used to save the trained model as a binary file to disk, which can be loaded back into memory and used for making predictions.

By using the pickle module, we can save the trained weights and architecture of the model, as well as any other necessary information such as the tokenizer or label encoder, to disk. This can be helpful when we want to deploy the model in a production environment or share it with other developers.

Overall, the pickle module is a useful tool in deep learning for saving and loading trained models, making it a convenient way to use the models in real-world applications.

In [75]:
import pickle
pickle.dump({'words':words, 'classes':classes, 'x_train':x_train, 'y_train':y_train}, open('training_data', 'wb'))

# Making the model

In [76]:
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

In [77]:
# Reading intent file again
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [78]:
# Loading the saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\acer\Deep Learning\Chat bot\model.tflearn


In [87]:
def clean_up_sentence(sentence):
    sentence_word = nltk.word_tokenize(sentence)    # Tokenizing the pattern
    sentence_word = [stemmer.stem(word.lower()) for word in sentence_word]    # Stemming each word
    return sentence_word

# Returning bag of word(bow) array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    sentence_word = clean_up_sentence(sentence)  # tokenizing the pattern
    
    # Generating bag of words
    bag = [0]*len(words)
    for s in sentence_word:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print('found in bag: %s'% w)
    
    return(np.array(bag))

In [88]:
ERROR_THRESHOLD = 0.30

def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]  # Generating probabilites from the model
    results = [[i,r] for i, r in enumerate(results) if r>ERROR_THRESHOLD]  # Filter out predictions below a Thershold value
    results.sort(key = lambda x: x[1], reverse=True)
    
    # Sort by strength of probability
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
        
    # Returning the tuple of intent and probability
    return return_list

def response(sentence, user_id='123', show_details=False):
    results = classify(sentence)
    if results:  # If we have classification then find the matching intent tag
        while results:  # Loop as long as there are matches to process
            for i in intents['intents']:  # find the tag matching the first result
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
            
            results.pop(0)

In [89]:
classify('What are you hours of operation?')

[('hours', 0.97457707)]

In [95]:
response('What are you hours of operation?')

Our hours are 9am-9pm every day


In [96]:
response('What is menu for today?')

Our speciality for today is Chicken Tikka


In [97]:
#Some of other context free responses.
response('Do you accept Credit Card?')

We accept most major credit cards


In [98]:
response('Where can we locate you?')

We are on the intersection of London Alley and Bridge Avenue.


In [99]:
response('That is helpful')

Any time!


In [100]:
response('Bye')

Have a nice day


In [101]:
#Adding some context to the conversation i.e. Contexualization for altering question and intents etc.
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [103]:
response('Can you please let me know the delivery options?')

We have home delivery options through UBER Eats and Zomato


In [104]:
response('What is menu for today?')

Our speciality for today is Chicken Tikka


In [105]:
context

{'123': 'food'}

In [106]:
response("Hi there!", show_details=True)

context: 
tag: greeting
Hello, thanks for visiting


In [107]:
response('What is menu for today?')

Today's special is Chicken Tikka
